In [7]:
# Imports
import pandas as pd  # type: ignore
import numpy as np  # type: ignore
import base64
from utils import (
    get_rules_list,
    create_train_test_split,
    create_model,
    payload_to_vec,
    create_adv_train_test_split,
)
from modsec import init_modsec

from sklearn.ensemble import RandomForestClassifier  # type: ignore
from wafamole.evasion import EvasionEngine  # type: ignore

In [8]:
# Set up variables

attack_data_path = "data/attacks_20k.sql"
sane_data_path = "data/sanes_20k.sql"

rule_ids = get_rules_list()
modsec = init_modsec()

In [9]:
# Create train and test sets and train model

paranoia_level = 4

# train, test = create_train_test_split(
#     attack_file=attack_data_path,
#     sane_file=sane_data_path,
#     train_attacks_size=2500,  # paper uses 10000
#     train_sanes_size=2500,  # paper uses 10000
#     test_attacks_size=500,  # paper uses 2000
#     test_sanes_size=500,  # paper uses 2000
#     modsec=modsec,
#     rule_ids=rule_ids,
#     paranoia_level=paranoia_level,
# )
# train.to_csv("data/train_5k.csv", index=False)
# test.to_csv("data/test_1k.csv", index=False)

# load the train and test sets from disk
train = pd.read_csv("data/train_5k.csv")
test = pd.read_csv("data/test_1k.csv")
train['vector'] = train['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
test['vector'] = test['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

wafamole_model, threshold = create_model(
    train=train,
    test=test,
    model=RandomForestClassifier(n_estimators=160, random_state=666),
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.97      0.94      0.95       500
        sane       0.94      0.97      0.95       500

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000

Adjusted threshold: 0.8027458911979671
              precision    recall  f1-score   support

      attack       0.85      0.99      0.92       500
        sane       0.99      0.83      0.90       500

    accuracy                           0.91      1000
   macro avg       0.92      0.91      0.91      1000
weighted avg       0.92      0.91      0.91      1000



In [10]:
# adversarial training

engine = EvasionEngine(wafamole_model)
train_adv, test_adv = create_adv_train_test_split(
    train=train,
    test=test,
    train_adv_size=50,
    test_adv_size=50,
    engine=engine,
    engine_settings={
        "max_rounds": 200,
        "round_size": 10,
        "timeout": 10,
        "threshold": threshold,
    },
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)
train_adv.to_csv("data/train_adv_50.csv", index=False)
test_adv.to_csv("data/test_adv_50.csv", index=False)

# train_adv = pd.read_csv("data/train_adv_50.csv")
# test_adv = pd.read_csv("data/test_adv_50.csv")
# train_adv['vector'] = train_adv['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))
# test_adv['vector'] = test_adv['vector'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

# train new model with train + train_adv
wafamole_model_adv, threshold_adv = create_model(
    train=pd.concat([train, train_adv]).sample(frac=1).reset_index(drop=True),
    test=pd.concat([test, test_adv]).sample(frac=1).reset_index(drop=True),
    model=RandomForestClassifier(n_estimators=160, random_state=666),
    desired_fpr=0.01,
    modsec=modsec,
    rule_ids=rule_ids,
    paranoia_level=paranoia_level,
)

Optimizing payloads...


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [06:07<00:00,  7.36s/it]


Creating vectors...


Processing payloads: 100%|██████████| 50/50 [00:06<00:00,  7.36it/s]


Model trained successfully!
Evaluating model...
Default threshold: 0.5
              precision    recall  f1-score   support

      attack       0.97      0.94      0.95       550
        sane       0.93      0.97      0.95       500

    accuracy                           0.95      1050
   macro avg       0.95      0.95      0.95      1050
weighted avg       0.95      0.95      0.95      1050

Adjusted threshold: 0.80625
              precision    recall  f1-score   support

      attack       0.86      0.99      0.92       550
        sane       0.99      0.83      0.90       500

    accuracy                           0.91      1050
   macro avg       0.92      0.91      0.91      1050
weighted avg       0.92      0.91      0.91      1050



In [11]:
# Test the model (without adversarial training)
engine = EvasionEngine(wafamole_model)

payload = 'SELECT SLEEP(5)#";'  # attack

payload_base64 = base64.b64encode(payload.encode("utf-8")).decode("utf-8")
vec = payload_to_vec(payload_base64, rule_ids, modsec, paranoia_level)
is_attack = wafamole_model.classify(payload)
print(f"Payload: {payload}")
print(f"Vec: {vec}")
print(f"Confidence: {round(is_attack, 5)}")

min_confidence, min_payload = engine.evaluate(
    payload=payload,
    max_rounds=200,
    round_size=10,
    timeout=60,
    threshold=0.0,
)
print()
print(f"Min payload: {min_payload.encode('utf-8')}")
print(f"Min confidence: {round(min_confidence, 5)}")
print(
    f"Reduced confidence from {round(is_attack, 5)} to {round(min_confidence, 5)} (reduction of {round(is_attack - min_confidence, 5)})"
)

print("\nEvasion successful" if min_confidence < threshold else "Evasion failed")

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.96247
[!] Execution timed out
Reached confidence 0.6761472509808204
with payload
sElect/*`a*/sLeEp((SELECT 5))  AND  0x1#";

Min payload: b'sElect/*`a*/sLeEp((SELECT 5))  AND  0x1#";'
Min confidence: 0.67615
Reduced confidence from 0.96247 to 0.67615 (reduction of 0.28632)

Evasion successful


In [13]:
# Test the model (with adversarial training)
engine_adv = EvasionEngine(wafamole_model_adv)

payload = 'SELECT SLEEP(5)#";'  # attack

payload_base64 = base64.b64encode(payload.encode("utf-8")).decode("utf-8")
vec = payload_to_vec(payload_base64, rule_ids, modsec, paranoia_level)
is_attack = wafamole_model_adv.classify(payload)
print(f"Payload: {payload}")
print(f"Vec: {vec}")
print(f"Confidence: {round(is_attack, 5)}")

min_confidence, min_payload = engine_adv.evaluate(
    payload=payload,
    max_rounds=200,
    round_size=10,
    timeout=60,
    threshold=0.0,
)
print()
print(f"Min payload: {min_payload.encode('utf-8')}")
print(f"Min confidence: {round(min_confidence, 5)}")
print(
    f"Reduced confidence from {round(is_attack, 5)} to {round(min_confidence, 5)} (reduction of {round(is_attack - min_confidence, 5)})"
)

print("\nEvasion successful" if min_confidence < threshold_adv else "Evasion failed")

Payload: SELECT SLEEP(5)#";
Vec: [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0]
Confidence: 0.97372
[!] Execution timed out
Reached confidence 0.6872175659600135
with payload
SEleCT
Sleep((SelEct (SeLecT 0X5)))||0#";0X5/Q+

Min payload: b'SEleCT\nSleep((SelEct (SeLecT 0X5)))||0#";0X5/Q+'
Min confidence: 0.68722
Reduced confidence from 0.97372 to 0.68722 (reduction of 0.2865)

Evasion successful
